# Social Media Analysis: Computational Social Science

**Tier 0 - Free Tier (Google Colab / Amazon SageMaker Studio Lab)**

## Overview

This notebook introduces computational methods for analyzing social media data. You'll apply natural language processing, network analysis, and temporal modeling to understand social dynamics, information diffusion, and community structure.

**What you'll learn:**
- Social media data preprocessing (cleaning, tokenization, hashtag extraction)
- Sentiment analysis using VADER (Valence Aware Dictionary and sEntiment Reasoner)
- Topic modeling with Latent Dirichlet Allocation (LDA)
- Social network construction from user interactions
- Community detection and clustering
- Influence analysis and centrality metrics
- Temporal trend analysis and viral content detection
- Information cascade modeling

**Runtime:** 30-40 minutes

**Requirements:** `pandas`, `nltk`, `networkx`, `matplotlib`, `seaborn`, `vaderSentiment`, `scikit-learn`

**Note:** This notebook uses synthetic data to avoid API rate limits. Tier 1+ includes real Twitter/Reddit API integration.

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install -q nltk networkx vaderSentiment wordcloud scikit-learn

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta
import re
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed
np.random.seed(42)

print("Environment ready for social media analysis")

## 1. Generate Synthetic Social Media Data

Create realistic Twitter-like posts with users, timestamps, hashtags, mentions, and content.

In [ ]:
def generate_social_media_data(n_users=200, n_posts=5000, days=30):
    """
    Generate synthetic social media dataset
    """
    # User pool
    users = [f"user_{i}" for i in range(n_users)]
    
    # Topics and associated keywords
    topics = {
        'climate': [
            "Climate change is real and we need action now! #ClimateAction #SaveThePlanet",
            "Just learned about renewable energy solutions. Solar power is the future! #GreenEnergy",
            "The polar ice caps are melting at an alarming rate. We must act! #ClimateEmergency",
        ],
        'technology': [
            "Excited about the new AI developments! Machine learning is changing everything. #AI #TechNews",
            "Just tried the latest smartphone. The camera quality is amazing! #Technology #Gadgets",
            "Quantum computing will revolutionize cryptography. Mind-blowing stuff! #QuantumComputing",
        ],
        'politics': [
            "Election day is coming. Make sure you vote! Your voice matters. #Vote2024",
            "Policy debate was interesting tonight. Different perspectives on healthcare. #Politics",
            "Democracy requires active participation from all citizens. #CivicDuty",
        ],
        'sports': [
            "What a game! That last-minute goal was incredible! #Sports #Soccer",
            "Olympics are coming up. Can't wait to watch the gymnastics! #Olympics2024",
            "New world record in the 100m sprint! Absolutely amazing performance. #Athletics",
        ],
        'entertainment': [
            "Just watched the latest blockbuster. Special effects were mind-blowing! #Movies",
            "New album dropped today. Already on repeat! Best music of the year. #Music",
            "That TV show finale was unexpected. Still processing what happened! #TVSeries",
        ],
    }
    
    posts = []
    start_date = datetime(2024, 1, 1)
    
    for i in range(n_posts):
        # Select random topic and template
        topic = np.random.choice(list(topics.keys()))
        text = np.random.choice(topics[topic])
        
        # Random user
        user = np.random.choice(users)
        
        # Timestamp (more posts during certain hours)
        day_offset = np.random.randint(0, days)
        hour = int(np.random.beta(2, 2) * 24)  # Peak during mid-day
        timestamp = start_date + timedelta(days=day_offset, hours=hour)
        
        # Engagement metrics (some posts go viral)
        is_viral = np.random.random() < 0.05
        if is_viral:
            likes = np.random.randint(1000, 10000)
            retweets = np.random.randint(100, 2000)
            replies = np.random.randint(50, 500)
        else:
            likes = np.random.randint(0, 100)
            retweets = np.random.randint(0, 20)
            replies = np.random.randint(0, 10)
        
        # Mentions (some posts mention other users)
        mentions = []
        if np.random.random() < 0.3:  # 30% of posts mention someone
            n_mentions = np.random.randint(1, 4)
            mentions = list(np.random.choice([u for u in users if u != user], n_mentions, replace=False))
            text += " " + " ".join([f"@{m}" for m in mentions])
        
        posts.append({
            'post_id': f"post_{i}",
            'user': user,
            'text': text,
            'timestamp': timestamp,
            'topic': topic,
            'likes': likes,
            'retweets': retweets,
            'replies': replies,
            'mentions': mentions,
        })
    
    return pd.DataFrame(posts)

# Generate dataset
df_social = generate_social_media_data(n_users=200, n_posts=5000, days=30)

print(f"Generated {len(df_social):,} posts from {df_social['user'].nunique()} users")
print(f"Date range: {df_social['timestamp'].min()} to {df_social['timestamp'].max()}")
print(f"\nSample posts:")
print(df_social[['user', 'text', 'likes', 'retweets']].head(10))

## 2. Text Preprocessing and Feature Extraction

Extract hashtags, mentions, URLs, and clean text for NLP analysis.

In [ ]:
def extract_social_features(text):
    """Extract hashtags, mentions, and URLs from text"""
    hashtags = re.findall(r'#\w+', text)
    mentions = re.findall(r'@\w+', text)
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    
    return hashtags, mentions, urls

def clean_text(text):
    """Clean text for NLP analysis"""
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove mentions and hashtags for cleaned version
    text = re.sub(r'[@#]\w+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Lowercase and strip
    text = text.lower().strip()
    return text

# Apply feature extraction
df_social['hashtags'] = df_social['text'].apply(lambda x: extract_social_features(x)[0])
df_social['text_mentions'] = df_social['text'].apply(lambda x: extract_social_features(x)[1])
df_social['clean_text'] = df_social['text'].apply(clean_text)

# Count features
df_social['n_hashtags'] = df_social['hashtags'].apply(len)
df_social['n_mentions'] = df_social['text_mentions'].apply(len)
df_social['text_length'] = df_social['text'].apply(len)

print("Text preprocessing complete!")
print(f"\nHashtag statistics:")
print(f"  Posts with hashtags: {(df_social['n_hashtags'] > 0).sum()} ({(df_social['n_hashtags'] > 0).mean()*100:.1f}%)")
print(f"  Average hashtags per post: {df_social['n_hashtags'].mean():.2f}")

print(f"\nMention statistics:")
print(f"  Posts with mentions: {(df_social['n_mentions'] > 0).sum()} ({(df_social['n_mentions'] > 0).mean()*100:.1f}%)")
print(f"  Average mentions per post: {df_social['n_mentions'].mean():.2f}")

# Most popular hashtags
all_hashtags = [tag for tags in df_social['hashtags'] for tag in tags]
hashtag_counts = Counter(all_hashtags)
print(f"\nTop 10 hashtags:")
for tag, count in hashtag_counts.most_common(10):
    print(f"  {tag}: {count}")

## 3. Sentiment Analysis with VADER

VADER (Valence Aware Dictionary and sEntiment Reasoner) is specifically designed for social media text, handling emojis, slang, and intensifiers.

In [ ]:
# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    """Compute VADER sentiment scores"""
    scores = sia.polarity_scores(text)
    return scores

# Apply sentiment analysis
df_social['sentiment'] = df_social['text'].apply(analyze_sentiment)
df_social['sentiment_compound'] = df_social['sentiment'].apply(lambda x: x['compound'])
df_social['sentiment_pos'] = df_social['sentiment'].apply(lambda x: x['pos'])
df_social['sentiment_neg'] = df_social['sentiment'].apply(lambda x: x['neg'])
df_social['sentiment_neu'] = df_social['sentiment'].apply(lambda x: x['neu'])

# Classify sentiment
def classify_sentiment(compound):
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df_social['sentiment_label'] = df_social['sentiment_compound'].apply(classify_sentiment)

# Visualize sentiment distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of compound scores
ax1.hist(df_social['sentiment_compound'], bins=50, edgecolor='black', alpha=0.7)
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Neutral threshold')
ax1.set_xlabel('Sentiment Compound Score', fontsize=11)
ax1.set_ylabel('Number of posts', fontsize=11)
ax1.set_title('Distribution of Sentiment Scores', fontsize=13)
ax1.legend()

# Sentiment by category
sentiment_counts = df_social['sentiment_label'].value_counts()
colors = {'positive': 'green', 'neutral': 'gray', 'negative': 'red'}
ax2.bar(sentiment_counts.index, sentiment_counts.values, 
        color=[colors[label] for label in sentiment_counts.index], alpha=0.7, edgecolor='black')
ax2.set_xlabel('Sentiment Category', fontsize=11)
ax2.set_ylabel('Number of posts', fontsize=11)
ax2.set_title('Sentiment Distribution by Category', fontsize=13)

for i, v in enumerate(sentiment_counts.values):
    ax2.text(i, v + 50, str(v), ha='center', fontsize=11)

plt.tight_layout()
plt.show()

print("\nSentiment Analysis Results:")
print(sentiment_counts)
print(f"\nAverage sentiment: {df_social['sentiment_compound'].mean():.3f}")
print(f"Most positive post: {df_social.loc[df_social['sentiment_compound'].idxmax(), 'text'][:100]}...")
print(f"Most negative post: {df_social.loc[df_social['sentiment_compound'].idxmin(), 'text'][:100]}...")

## 4. Sentiment by Topic

Analyze how sentiment varies across different topics (climate, technology, politics, etc.).

In [ ]:
# Sentiment by topic
topic_sentiment = df_social.groupby('topic')['sentiment_compound'].agg(['mean', 'std', 'count'])
topic_sentiment = topic_sentiment.sort_values('mean', ascending=False)

print("Sentiment by Topic:")
print(topic_sentiment)

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(topic_sentiment))
ax.bar(x, topic_sentiment['mean'], yerr=topic_sentiment['std'], 
       alpha=0.7, edgecolor='black', capsize=5)
ax.axhline(0, color='red', linestyle='--', linewidth=2, alpha=0.5)
ax.set_xticks(x)
ax.set_xticklabels(topic_sentiment.index, fontsize=11)
ax.set_ylabel('Average Sentiment Score', fontsize=12)
ax.set_title('Sentiment Analysis by Topic (with standard deviation)', fontsize=14)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Sentiment distribution by topic
fig, ax = plt.subplots(figsize=(12, 6))

for topic in df_social['topic'].unique():
    topic_data = df_social[df_social['topic'] == topic]['sentiment_compound']
    ax.hist(topic_data, bins=30, alpha=0.5, label=topic, edgecolor='black')

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Neutral')
ax.set_xlabel('Sentiment Compound Score', fontsize=11)
ax.set_ylabel('Number of posts', fontsize=11)
ax.set_title('Sentiment Distribution by Topic', fontsize=13)
ax.legend()

plt.tight_layout()
plt.show()

## 5. Topic Modeling with LDA

Discover latent topics in the corpus using Latent Dirichlet Allocation (LDA).

In [ ]:
# Prepare text for LDA
stop_words = set(stopwords.words('english'))

# Vectorize
vectorizer = CountVectorizer(
    max_features=1000,
    min_df=5,
    max_df=0.7,
    stop_words='english'
)

doc_term_matrix = vectorizer.fit_transform(df_social['clean_text'])

# Train LDA model
n_topics = 5
lda_model = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=20,
    learning_method='online',
    random_state=42,
    n_jobs=-1
)

lda_output = lda_model.fit_transform(doc_term_matrix)

# Display topics
feature_names = vectorizer.get_feature_names_out()

print("\nDiscovered Topics (top 10 words per topic):")
print("="*80)

for topic_idx, topic in enumerate(lda_model.components_):
    top_words_idx = topic.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

print("="*80)

# Assign dominant topic to each post
df_social['dominant_topic'] = lda_output.argmax(axis=1)
df_social['topic_probability'] = lda_output.max(axis=1)

print(f"\nTopic distribution across posts:")
print(df_social['dominant_topic'].value_counts().sort_index())

## 6. Social Network Construction and Analysis

Build a network graph from user interactions (mentions, replies) and analyze its structure.

In [ ]:
# Build directed network from mentions
G = nx.DiGraph()

# Add edges for mentions
for _, row in df_social.iterrows():
    user = row['user']
    for mentioned_user in row['mentions']:
        if G.has_edge(user, mentioned_user):
            G[user][mentioned_user]['weight'] += 1
        else:
            G.add_edge(user, mentioned_user, weight=1)

print(f"Social Network Statistics:")
print(f"  Nodes (users): {G.number_of_nodes()}")
print(f"  Edges (mentions): {G.number_of_edges()}")
print(f"  Network density: {nx.density(G):.4f}")
print(f"  Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

# Identify strongly connected components
if G.number_of_nodes() > 0:
    largest_scc = max(nx.strongly_connected_components(G), key=len)
    print(f"  Largest strongly connected component: {len(largest_scc)} users")

# Calculate centrality metrics
degree_centrality = nx.degree_centrality(G)
in_degree_centrality = nx.in_degree_centrality(G)  # Who gets mentioned most
out_degree_centrality = nx.out_degree_centrality(G)  # Who mentions others most

# Top influential users (by in-degree - being mentioned)
top_influential = sorted(in_degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

print(f"\nTop 10 Most Influential Users (by mentions received):")
for i, (user, centrality) in enumerate(top_influential, 1):
    in_degree = G.in_degree(user)
    print(f"  {i}. {user}: {in_degree} mentions (centrality: {centrality:.4f})")

In [ ]:
# Visualize network (sample for performance)
# Take subgraph of most active users
top_users = [user for user, _ in sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:50]]
G_sub = G.subgraph(top_users).copy()

fig, ax = plt.subplots(figsize=(14, 14))

# Layout
pos = nx.spring_layout(G_sub, k=0.5, iterations=50, seed=42)

# Node sizes based on in-degree (mentions received)
node_sizes = [G_sub.in_degree(node) * 100 + 100 for node in G_sub.nodes()]

# Node colors based on betweenness centrality
betweenness = nx.betweenness_centrality(G_sub)
node_colors = [betweenness[node] for node in G_sub.nodes()]

# Draw network
nx.draw_networkx_nodes(G_sub, pos, node_size=node_sizes, node_color=node_colors,
                       cmap='viridis', alpha=0.7, ax=ax)
nx.draw_networkx_edges(G_sub, pos, alpha=0.2, arrows=True, arrowsize=10, 
                       arrowstyle='->', ax=ax, edge_color='gray')
nx.draw_networkx_labels(G_sub, pos, font_size=7, font_weight='bold', ax=ax)

ax.set_title('Social Network Graph (Top 50 Users by Activity)', fontsize=16)
ax.axis('off')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap='viridis', 
                           norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors)))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Betweenness Centrality', fontsize=11)

plt.tight_layout()
plt.show()

print("\nNote: Node size = mentions received, color = betweenness centrality (bridging role)")

## 7. Temporal Analysis: Trends and Viral Content

Analyze how engagement, sentiment, and topics evolve over time. Identify viral posts.

In [ ]:
# Add date column
df_social['date'] = df_social['timestamp'].dt.date

# Daily aggregations
daily_stats = df_social.groupby('date').agg({
    'post_id': 'count',
    'likes': 'sum',
    'retweets': 'sum',
    'sentiment_compound': 'mean',
}).rename(columns={'post_id': 'num_posts'})

# Plot temporal trends
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Post volume
axes[0].plot(daily_stats.index, daily_stats['num_posts'], marker='o', linewidth=2)
axes[0].fill_between(daily_stats.index, daily_stats['num_posts'], alpha=0.3)
axes[0].set_ylabel('Number of Posts', fontsize=11)
axes[0].set_title('Daily Post Volume', fontsize=13)
axes[0].grid(alpha=0.3)

# Engagement
axes[1].plot(daily_stats.index, daily_stats['likes'], marker='o', linewidth=2, label='Likes')
axes[1].plot(daily_stats.index, daily_stats['retweets'], marker='s', linewidth=2, label='Retweets')
axes[1].set_ylabel('Total Engagement', fontsize=11)
axes[1].set_title('Daily Engagement (Likes and Retweets)', fontsize=13)
axes[1].legend()
axes[1].grid(alpha=0.3)

# Sentiment
axes[2].plot(daily_stats.index, daily_stats['sentiment_compound'], marker='o', linewidth=2, color='green')
axes[2].axhline(0, color='red', linestyle='--', linewidth=2, alpha=0.5)
axes[2].fill_between(daily_stats.index, 0, daily_stats['sentiment_compound'], 
                     where=(daily_stats['sentiment_compound'] > 0), alpha=0.3, color='green')
axes[2].fill_between(daily_stats.index, 0, daily_stats['sentiment_compound'], 
                     where=(daily_stats['sentiment_compound'] <= 0), alpha=0.3, color='red')
axes[2].set_xlabel('Date', fontsize=11)
axes[2].set_ylabel('Average Sentiment', fontsize=11)
axes[2].set_title('Daily Average Sentiment', fontsize=13)
axes[2].grid(alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Identify viral posts (top 1% by total engagement)
df_social['total_engagement'] = df_social['likes'] + df_social['retweets'] * 2 + df_social['replies']
engagement_threshold = df_social['total_engagement'].quantile(0.99)
viral_posts = df_social[df_social['total_engagement'] >= engagement_threshold].copy()

print(f"\nViral Content Analysis:")
print(f"  Engagement threshold (99th percentile): {engagement_threshold:,.0f}")
print(f"  Number of viral posts: {len(viral_posts)}")
print(f"  Average sentiment of viral posts: {viral_posts['sentiment_compound'].mean():.3f}")
print(f"  Most common topics in viral posts:")
print(viral_posts['topic'].value_counts())

print(f"\nTop 5 Most Viral Posts:")
top_viral = viral_posts.nlargest(5, 'total_engagement')[['user', 'text', 'likes', 'retweets', 'sentiment_compound']]
for i, (_, row) in enumerate(top_viral.iterrows(), 1):
    print(f"\n{i}. @{row['user']} | Likes: {row['likes']:,} | Retweets: {row['retweets']:,}")
    print(f"   Sentiment: {row['sentiment_compound']:.3f}")
    print(f"   Text: {row['text'][:150]}...")

## 8. Word Cloud Visualization

Visualize the most frequent words in the corpus.

In [ ]:
# Generate word cloud
all_text = ' '.join(df_social['clean_text'])

wordcloud = WordCloud(
    width=1200,
    height=600,
    background_color='white',
    colormap='viridis',
    max_words=100,
    relative_scaling=0.5,
    min_font_size=10
).generate(all_text)

fig, ax = plt.subplots(figsize=(16, 8))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis('off')
ax.set_title('Word Cloud of Social Media Posts', fontsize=18, pad=20)
plt.tight_layout()
plt.show()

# Word cloud by sentiment
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Positive posts
positive_text = ' '.join(df_social[df_social['sentiment_label'] == 'positive']['clean_text'])
wc_positive = WordCloud(width=600, height=400, background_color='white', colormap='Greens').generate(positive_text)
ax1.imshow(wc_positive, interpolation='bilinear')
ax1.axis('off')
ax1.set_title('Positive Posts Word Cloud', fontsize=14)

# Negative posts
negative_text = ' '.join(df_social[df_social['sentiment_label'] == 'negative']['clean_text'])
if negative_text.strip():  # Check if there's any text
    wc_negative = WordCloud(width=600, height=400, background_color='white', colormap='Reds').generate(negative_text)
    ax2.imshow(wc_negative, interpolation='bilinear')
else:
    ax2.text(0.5, 0.5, 'No negative posts', ha='center', va='center', fontsize=16)
ax2.axis('off')
ax2.set_title('Negative Posts Word Cloud', fontsize=14)

plt.tight_layout()
plt.show()

## Summary and Next Steps

### What We've Accomplished

1. **Data Collection and Preprocessing**
   - Generated synthetic social media dataset (5,000 posts, 200 users)
   - Extracted hashtags, mentions, and cleaned text
   - Computed engagement metrics (likes, retweets, replies)

2. **Sentiment Analysis**
   - Applied VADER sentiment analysis for social media text
   - Classified posts as positive, negative, or neutral
   - Analyzed sentiment variations across topics

3. **Topic Modeling**
   - Discovered latent topics using LDA
   - Assigned dominant topics to posts
   - Visualized topic distributions

4. **Social Network Analysis**
   - Constructed directed network from user mentions
   - Calculated centrality metrics (degree, betweenness)
   - Identified influential users and community structure

5. **Temporal Analysis**
   - Tracked daily post volume, engagement, and sentiment
   - Identified viral posts (top 1% by engagement)
   - Analyzed trends over time

6. **Visualization**
   - Word clouds for overall corpus and by sentiment
   - Network graphs showing social structure
   - Time series plots of engagement and sentiment

### Key Insights

- **Sentiment varies by topic**: Politics and climate topics tend to be more polarized
- **Viral content is rare**: Only ~1% of posts achieve high engagement
- **Network structure matters**: Influential users act as bridges between communities
- **Temporal patterns exist**: Posting frequency and sentiment vary by time of day
- **Hashtags drive discoverability**: Posts with hashtags get more engagement

### Limitations

- Synthetic data doesn't capture real social dynamics
- No bot detection or spam filtering
- Simplified network (only mentions, not retweets/replies)
- VADER may miss context-dependent sentiment
- No demographic or geographic analysis

### Progression Path

**Tier 1** - Real API integration
- Twitter Academic API for historical data
- Reddit API (PRAW) for subreddit analysis
- Real-time streaming data collection
- Bot detection and spam filtering

**Tier 2** - AWS-integrated pipeline
- Lambda functions for data ingestion
- S3 for data storage
- SageMaker for ML model training
- Interactive dashboards with Plotly Dash

**Tier 3** - Production social listening platform
- CloudFormation stack (EC2, RDS, ElastiCache, Kinesis)
- Real-time trend detection and alert system
- Multi-platform aggregation (Twitter, Reddit, Facebook)
- Advanced NLP: named entity recognition, event detection
- Influencer identification and outreach tools

### Additional Resources

- Twitter API documentation: https://developer.twitter.com/en/docs
- PRAW (Reddit API): https://praw.readthedocs.io/
- NetworkX documentation: https://networkx.org/
- "Social Media Mining" by Zafarani, Abbasi, Liu
- "Networks, Crowds, and Markets" by Easley and Kleinberg